# Expanding on Ito et al. (2017) to recover simulated task activity and connectivity matrix


Set up environment

In [ ]:
from collections import OrderedDict
import copy
import sys
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import pandas as pd
import seaborn as sns
sns.set_style("white")
import scipy.stats as stats
import statsmodels.api as sm

Import custom modules

In [ ]:
sys.path.append('../')
sys.path.append('../../utils/')

# Primary module with most model functions
import model

# Module for FC regression
import multregressionconnectivity as mreg

Helper functions

In [ ]:
phi = lambda x: np.tanh(x)

inv_phi = lambda x: np.arctanh(x)

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)


## Neural network model setup

In this notebook we expand on simulations described in [Ito et al. (2017)](https://www.nature.com/articles/s41467-017-01000-w.pdf). Ito et al. propose a dynamic neural network model to simulate resting state and task data. Accordingly change in activity in each node is a function of the local connectivity determined by $s$, global connectivity determined by $g$ and task activity for that node described in $I$

\begin{equation*}
\frac{dx_i}{dt}\tau_i = -x_i(t) + s\phi\big(x_i(t)\big) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(x_j(t)\big)\Bigg) + I_i(t)
\end{equation*}

where transformation function $\phi$ is
\begin{equation*}
\phi(x) = \tanh(x) = \frac{\sinh(x)}{\cosh(x)} = \frac{e^{2x}-1}{e^{2x}+1}
\end{equation*}

This transformation is intended to introduce *"a nonlinearity to the interactions among units that is
similar to aggregate nonlinearity from neuronal action potentials"* as explained in [Cole et al. (2016)](https://www.nature.com/articles/nn.4406.pdf). The effect of this transformation can be seen below in the attenuated signal of the transformed timeseries.

In [ ]:
plt.rcParams["figure.figsize"][0] = 6
plt.rcParams["figure.figsize"][1] = 5
a = np.random.normal(size=100)
pa = phi(a)
plt.plot(a, label="Untransformed")
plt.plot(pa, label="Transformed")
plt.axhline(y=1,linewidth=2, color='gray', ls = "--")
plt.axhline(y=-1,linewidth=2, color='gray', ls = "--")
plt.legend()

In Ito et al. (2017) $\tau_i$, $s$, $g$ and $dt$ are set to 1. This simplifies the equation to:

\begin{equation*}
x_i(t) + \frac{dx_i}{dt} = \phi\big(x_i(t)\big) + \sum_{j\neq i}^{N} W_{ij}\phi\big(x_j(t)\big) + I_i(t)
\end{equation*}

***Create structural and synaptic network***

One hub and two local networks

In [ ]:
# Parameters for subject's networks
ncommunities = 3
innetwork_dsity = .60
outnetwork_dsity = .08
hubnetwork_dsity = .25

nodespercommunity = 35
totalnodes = nodespercommunity*ncommunities

In [ ]:
# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

In [ ]:
plt.rcParams["figure.figsize"][0] = 5
plt.rcParams["figure.figsize"][1] = 4
sns.heatmap(W, xticklabels=False, yticklabels=False)
plt.xlabel('Regions')
plt.ylabel('Regions')
plt.title("Synaptic Weight Matrix -- Coupling Matrix")

Note in this matrix:

`W[..., x]` : column x of matrix denotes all outgoing connection weights from node x  
`W[x, ...]` : row x of matrix denotes all incoming connection weights to node x

In [ ]:
## Set up simulation parameters
dt = 1.0
tau = 1.0
g = 1.0
s = 1.0

### Task activity 

***Hub network stimulation***

In [ ]:
Ci = np.repeat(np.arange(ncommunities),nodespercommunity) # Construct a community affiliation vector
hub_ind = np.where(Ci==0)[0] # Identify the regions associated with the hub network (hub network is by default the 0th network)

# Set number of time points for each task
Tmax = 100000 

T = np.arange(0,Tmax,dt)
    
# Construct timing array for convolution -- this timing is irrespective of the task being performed
# Tasks are only determined by which nodes are stimulated
tasktiming = np.zeros((1,len(T)))
for t in range(len(T)):
    if t%2000>500 and t%2000<1000:
        tasktiming[0,t] = 1.0

Block design where stimulus is ON for 500 ms every 2 seconds looks like:

In [ ]:
plt.plot(T[:10000], tasktiming[0,:10000])
plt.ylim(top = 1.2, bottom = -0.1)

In [ ]:
stimtimes = {}

topdown_only = range(1,2)
task = 1

taskcount = task-np.min(topdown_only)
stimsize = np.floor(nodespercommunity/3.0)
stim_nodes = np.arange((taskcount)*stimsize,(taskcount)*stimsize+stimsize,dtype=int)
stimtimes[task] = np.zeros((totalnodes,len(T)))

# When task is ON the activity for a stim_node at that time point is .5
for t in range(len(T)):
    if tasktiming[0,t] == 1:
        stimtimes[task][stim_nodes,t] = .5

Activated nodes in the hub network *only* (making it a top-down task) are:

In [ ]:
stim_nodes

`taskdata` is an $n$ by $t$ matrix containing the timeseries for each node where $n$-rows are for nodes and $t$ columns are for the time points.

It is generated by updating each column (each time point) for **all nodes**  

At each time point the differential equation describing the change in activity is solved using the [Runge-Kutta second order method](https://lpsa.swarthmore.edu/NumInt/NumIntSecond.html). Runge-Kutta methods are used to discretize the problem of updating values for continuous time when solving differential equations. Ito et al. hypothesize the differential equation determining the amount of change in each timestep as:

\begin{equation*}
\frac{dx_i}{dt} = \frac{-x_i(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(x_j(t)\big)\Bigg) + s\phi\big(x_i(t)\big) + I_i(t)}{\tau_i}
\end{equation*}

Thus `taskdata` for all nodes $x_i$ consists of the current activity plus a weighted sum of approximate changes in activity calculated using the differential equation.

\begin{equation*}
x_{i}(t+dt) = x_{i}(t) + \frac{k1_{i}+k2_{i}}{2}
\end{equation*}

The first derivative based on $t(0)$ that is used for the approximation is

\begin{equation*}
k1_{i} = \frac{-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)}{\tau}
\end{equation*}

$\hat{I}_{i}(t)$ is the spontaneous activity that is the sum of randomly generated noise and task-related stimulation ($\hat{I}_{i}(t) = {I}_{i}(t)+\epsilon(t)$). Since the column-vector of task-related stimulation activity is very sparse ($|I_{i}(t)| \neq 0$ only for $i$ that is the third of the hub nodes and only for a short window of timepoints $t$) **most of the activity feeds into updating the activity of a node is noise (which in this framework is the equivalent of rest activity)**.

Using this slope the first intermediate approximation at the endpoint is

\begin{equation*}
x_{i}^*(t+dt) = x_{i}(t) + k1_{i} dt
\end{equation*}

and the second order approximation for the change in activity is

\begin{equation*}
k2_{i} = \frac{-x_{i}^*(t+dt) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(x_{i}^*(t+dt)\big) + \hat{I}_{i}(t+1)}{\tau}
\end{equation*}

If we discretize and work through the algebra the row-wise (node-centric) 'expanded' GLM for the column-wise (timepoint-centric) generated data using the Runge-Kutte method would be:

\begin{equation*}
x_{i}(t+1) = x_{i}(t) + dt\frac{k1_{i}+k2_{i}}{2}
\end{equation*}

\begin{equation*}
x_{i}(t+1) = x_{i}(t) + dt\frac{\frac{-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)}{\tau}+\frac{-x_{i}^*(t+dt) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(x_{i}^*(t+dt)\big) + \hat{I}_{i}(t+1)}{\tau}}{2}
\end{equation*}

Replacing $x_{i}^*(t+dt)$

\begin{equation*}
x_{i}(t+1) = x_{i}(t) + dt\frac{\frac{-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)}{\tau}+\frac{-x_{i}(t) - k1_{i} dt + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(x_{i}(t) + k1_{i} dt\big) + \hat{I}_{i}(t+1)}{\tau}}{2}
\end{equation*}

Getting ride of $s, g, \tau, dt$ (as they are 1 in these simulations) cleaning up and moving the denominator

\begin{equation*}
2x_{i}(t+1) = \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + \phi\big(x_{i}(t)\big) + \hat{I}_{i}(t) - k1_{i} + \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + \phi\big(x_{i}(t) + k1_{i} \big) + \hat{I}_{i}(t+1)
\end{equation*}

Replacing $k1_i$

\begin{equation*}
2x_{i}(t+1) = \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + \phi\big(x_{i}(t)\big) + \hat{I}_{i}(t) - \Bigg[-x_{i}(t) + \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + \phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg] + \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + \phi\big(x_{i}(t) + \Bigg[-x_{i}(t) + \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + \phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg]\big) + \hat{I}_{i}(t+1)
\end{equation*}

Cleanup

\begin{equation*}
2x_{i}(t+1) = x_{i}(t) + \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + \phi\Bigg(\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + \phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg) + \hat{I}_{i}(t+1)
\end{equation*}

We can use this equation to expand the traditinal GLM. A traditional GLM would have modeled

\begin{equation*}
x_{i}(t) = h(t) \circledast I(t) + \epsilon_i(t)
\end{equation*}

that is the activity of a voxel at a given time point would have been a function of the convolved task activity for that node at that time point plus some noise (For these simulations we ignore other movement confounds etc. included in level 1 models in real data).  

With the expanded model derived from the approximation of the differential equation we model the value at the next time step of a given node ($x_{i}(t+1)$) as a function of their activity in the current time step ($x_{i}(t)$), the transformed activity in the next time step from all other nodes weighted by their connection weights to the given node ($\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg)$), the transformed activity in the current time step resulting from other nodes, self and task ($\phi\Bigg(\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + \phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg)$) and task activity for that node in the next time step ($\hat{I}_{i}(t+1)$).

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]
#taskdata = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
#                                          I=stimtimes[task], noise=1)

## Extending GLM (eGLM) to recover true task activity

Note: Extending the GLM in this way is intended to account for the connectivity between nodes and how **task** activity in the previous step affects task activity in the current step for a given node. I don't think this is the same thing as 'subtracting out' resting state to look at the remaining task activity.


### Correcting task activity knowing the connectivity matrix

With real data we can estimate the weight matrix from resting state but we wouldn't know which nodes a task activates. Thus we wouldn't have access to $\hat{I}_i(t)$ but only to $\hat{I}(t)$, i.e. assume that the regressor has the same effect on all nodes. 

Running the model

\begin{equation*}
2x_{i}(t+1) = x_{i}(t) + \Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}(t+1)\big)\Bigg) + \phi\Bigg(\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}(t)\big)\Bigg) + \phi\big(x_{i}(t)\big) + \hat{I}(t)\Bigg) + \hat{I}(t+1)
\end{equation*}

In [ ]:
#Using first third of the data for speed of estimation
taskdata_short = copy.copy(taskdata[:,:15000])

In [ ]:
y = copy.copy(taskdata_short)
I = copy.copy(stimtimes[task][:,:15000])

In [ ]:
ntimepoints = y.shape[1]
nregions = y.shape[0]

#Initialize empty variables
ucr_task = np.zeros((nregions))
ext_cur_node = np.zeros((nregions)) 
ext_other_ns_next_spont = np.zeros((nregions)) 
ext_phi_others_cur_spont = np.zeros((nregions)) 
ext_task = np.zeros((nregions))


In [ ]:
for region in range(0, nregions):
    
    cur_y = y[region,:]
    incoming_connections = W[region, :]
    incoming_connections = np.delete(incoming_connections,region)
    #task_reg = I[region,:]
    task_reg = I[0,:]

    ucr_mod = sm.OLS(cur_y, task_reg)
    ucr_res = ucr_mod.fit()
    ucr_task[region] = ucr_res.params[0]

    #Extended model 
    next_y = 2*cur_y[1:] #shift column up to predict next multiplied activity

    cur_y = cur_y[:-1] #drop last time point

    drop_region = [region]
    other_ns_next_spont = np.delete(y, drop_region, axis=0)[:,1:] #dropping first col/timepoint
    other_ns_next_spont = other_ns_next_spont.T
    other_ns_next_spont = np.apply_along_axis(phi, 0, other_ns_next_spont)
    other_ns_next_spont = np.sum(other_ns_next_spont*incoming_connections, axis = 1)

    other_ns_cur_spont = np.delete(y, drop_region, axis=0)[:,:-1] #dropping last col/timepoint
    other_ns_cur_spont = other_ns_cur_spont.T
    other_ns_cur_spont = np.apply_along_axis(phi, 0, other_ns_cur_spont)
    other_ns_cur_spont = np.sum(other_ns_cur_spont*incoming_connections, axis = 1)

    cur_y_phi = phi(cur_y)

    cur_n_cur_spont = task_reg[:-1]
    
    phi_others_cur_spont = phi(other_ns_cur_spont + cur_y_phi + cur_n_cur_spont)
    
    cur_n_next_spont = task_reg[1:]

    ext_des_mat = np.concatenate((cur_y.reshape(-1,1), other_ns_next_spont.reshape(-1,1)), 1)
    ext_des_mat = np.concatenate((ext_des_mat, phi_others_cur_spont.reshape(-1,1)), 1)
    ext_des_mat = np.concatenate((ext_des_mat, cur_n_next_spont.reshape(-1,1)), 1)

    ext_mod = sm.OLS(next_y, ext_des_mat)
    ext_res = ext_mod.fit()
    ext_params = ext_res.params

    ext_cur_node[region] = ext_params[0]
    ext_other_ns_next_spont[region] = ext_params[1]
    ext_phi_others_cur_spont[region] = ext_params[2]
    ext_task[region] = ext_params[3]

#### Task betas reduced after correction?

Yes, but not the level of 'true' activity (0.5) for all activated nodes.  

Note the three levels in the classic GLM: true activity estimates bleed more into the hub network nodes compared to the local network nodes.

In [ ]:
stim_mag = 0.5
true_task_stim = np.zeros(totalnodes)
true_task_stim[stim_nodes] = stim_mag

In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", label = "Classic GLM")
plt.plot(ext_task, color = "green", label = "Extended GLM")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

The deviance in over- and under-estimaton for the stimulated vs. non-stimulated nodes in the extended GLM are not different from each other.

In [ ]:
plt.plot(abs(ucr_task-true_task_stim), color = "blue", label = "Classic GLM")
plt.plot(abs(ext_task-true_task_stim), color = "green", label = "Extended GLM")
plt.axhline(y=0,linewidth=2, color='gray', ls = "--")
plt.ylabel('|Estimate - True|',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

Check if  
1. The mean of the task coefficients estimated for non-stimulated nodes is not different from the true value of 0 
2. the mean of the task coefficients estimated for the stimulated nodes is not different from the true value 0.5

In [ ]:
stats.ttest_ind(np.delete(ext_task, stim_nodes), np.repeat(0, 94))

In [ ]:
stats.ttest_ind(ext_task[stim_nodes], np.repeat(0.5,11))

## Generalizable function for eGLM

The loop above that runs an extended GLM on each node's timeseries accounting for its local and global connectivity uses the simplified differential equation that depends on the set values of the parameters.  

But the parameters used in generating the neural network ($dt$, $\tau$, $g$, $s$) need not be 1. Therefore the extended GLM model would not always simplify to the equation above. To be able to test this method of extending the traditional GLM more generally pn different network and task structures we need to algebraically write out the full equation including all the parameters that describes how activity in the next time step of each node is updated ($x_{i}(t+1)$).


\begin{equation*}
k1_{i} = \frac{-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)}{\tau}
\end{equation*}

\begin{equation*}
x_{i}(t+1) = x_{i}(t) + dt\frac{\frac{-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)}{\tau}+\frac{-x_{i}(t) - k1_{i} dt + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(x_{i}(t) + k1_{i} dt\big) + \hat{I}_{i}(t+1)}{\tau}}{2}
\end{equation*}

To write out a general equation for $x_i(t+1)$ first we replace the $k1_i$ in the equation. Since $k1_i$ appears always in the context of $x_i(t)+k1_idt$ we plug in the formula for $k1_i$ into this expression and simplify

\begin{equation*}
x_{i}(t) + k1_{i}dt = x_{i}(t)+dt\frac{-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)}{\tau}
\end{equation*}

\begin{equation*}
x_{i}(t) + k1_{i}dt = \frac{\tau-dt}{\tau}x_{i}(t)+\frac{dt}{\tau}\Bigg[g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg]
\end{equation*}


Then we use this expression in the formula of $x_{i}(t+1)$ 

\begin{equation*}
x_{i}(t+1) = x_{i}(t) + \frac{dt}{2\tau}\Bigg[-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t) - \big(x_{i}(t) + k1_{i} dt\big) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(x_{i}(t) + k1_{i} dt\big) + \hat{I}_{i}(t+1)\Bigg]
\end{equation*}

Plugging in $x_{i}(t) + k1_{i}dt$

\begin{equation*}
x_{i}(t+1) = x_{i}(t) + \frac{dt}{2\tau}\Bigg[-x_{i}(t) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t) - \big(\frac{\tau-dt}{\tau}x_{i}(t)+\frac{dt}{\tau}\Bigg[g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg]\big) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(\frac{\tau-dt}{\tau}x_{i}(t)+\frac{dt}{\tau}\Bigg[g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg]\big) + \hat{I}_{i}(t+1)\Bigg]
\end{equation*}

Simplifying common terms

\begin{equation*}
x_{i}(t+1) = x_{i}(t) + \frac{dt}{2\tau}\Bigg[\frac{dt-2\tau}{\tau}x_{i}(t) + \frac{\tau-dt}{\tau}\Bigg[g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg] + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(\frac{\tau-dt}{\tau}x_{i}(t)+\frac{dt}{\tau}\Bigg[g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg]\big) + \hat{I}_{i}(t+1)\Bigg]
\end{equation*}

Extracting $x_{i}(t)$ outside the rest of the expression

\begin{equation*}
x_{i}(t+1) = \frac{2\tau^2+dt(dt-2\tau)}{2\tau^2}x_{i}(t) + \frac{dt}{2\tau}\Bigg[\frac{\tau-dt}{\tau}\Bigg(g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t+1)\big)\Bigg) + s\phi\big(\frac{\tau-dt}{\tau}x_{i}(t)+\frac{dt}{\tau}\Bigg(g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(\hat{I}_j(t)\big)\Bigg) + s\phi\big(x_{i}(t)\big) + \hat{I}_{i}(t)\Bigg)\big) + \hat{I}_{i}(t+1)\Bigg]
\end{equation*}

This is a pretty messy expression. In words it describes the activity in the next time point for a node as a function of:
- current node activity 
- current activity depending on connectivity with other nodes, local connectivity and task stimulation 
- activity in the next time step depending on connectivity with other nodes, first order approximation in the current node and task stimulation  
all of which are weigted by different constants that depends on the time steps

Since this is just the generalized form of the simplified extended GLM equation including all the terms that were previously omitted because their values were known to be 1 we should get the same results if we rewrite the code for the extended GLM with this equation.

In [ ]:
def run_ucr_glm(all_nodes_ts, task_reg):
    nregions = all_nodes_ts.shape[0]
    ucr_task_betas = np.zeros((nregions))
    
    for region in range(0, nregions):
        cur_y = all_nodes_ts[region,:]
        ucr_mod = sm.OLS(cur_y, task_reg)
        ucr_res = ucr_mod.fit()
        ucr_task_betas[region] = ucr_res.params[0]
    
    return ucr_task_betas

In [ ]:
def run_ext_glm(all_nodes_ts, task_reg, weight_matrix, dt, tau, g, s): 
    
    nregions = all_nodes_ts.shape[0]
    ext_task_betas = np.zeros((nregions))
    
    for region in range(0, nregions):
        cur_y = all_nodes_ts[region,:]
        incoming_connections = weight_matrix[region, :]
        incoming_connections = np.delete(incoming_connections,region)
        drop_region = [region]
        
        #DV
        next_y = cur_y[1:] #shift column up to predict activity in next time point

        #IV 1
        cur_y = cur_y[:-1] #drop last time point
        cur_y = ((2*(tau**2)+dt*(dt-2*tau))/(2*(tau**2)))*cur_y
        
        #IV 2
        other_ns_cur_spont = np.delete(y, drop_region, axis=0)[:,:-1] #dropping last col/timepoint
        other_ns_cur_spont = other_ns_cur_spont.T
        other_ns_cur_spont = np.apply_along_axis(phi, 0, other_ns_cur_spont)
        other_ns_cur_spont = np.sum(other_ns_cur_spont*incoming_connections, axis = 1) 
        other_ns_cur_spont = (dt/(2*tau))*((tau-dt)/tau)*g*other_ns_cur_spont
        
        #IV 3
        cur_y_phi = all_nodes_ts[region,:]
        cur_y_phi = cur_y_phi[:-1]
        cur_y_phi = (dt/(2*tau))*((tau-dt)/tau)*s*phi(cur_y_phi)
        
        #IV 4
        cur_n_task = (dt/(2*tau))*((tau-dt)/tau)*task_reg[:-1]
        
        #IV 5
        other_ns_next_spont = np.delete(y, drop_region, axis=0)[:,1:] #dropping first col/timepoint
        other_ns_next_spont = other_ns_next_spont.T
        other_ns_next_spont = np.apply_along_axis(phi, 0, other_ns_next_spont)
        other_ns_next_spont = np.sum(other_ns_next_spont*incoming_connections, axis = 1)
        other_ns_next_spont = (dt/(2*tau))*g*other_ns_next_spont
        
        #IV 6
        cur_n_first_appr = all_nodes_ts[region,:]
        cur_n_first_appr = cur_n_first_appr[:-1]
        cur_n_first_appr = ((tau-dt)/tau)*cur_n_first_appr
        tmp = np.delete(all_nodes_ts, drop_region, axis=0)[:,:-1] #dropping last col/timepoint
        tmp = tmp.T
        tmp = np.apply_along_axis(phi, 0, tmp)
        tmp = np.sum(tmp*incoming_connections, axis = 1) 
        tmp = (dt/tau)*g*tmp
        cur_n_first_appr = cur_n_first_appr+tmp
        tmp = all_nodes_ts[region,:]
        tmp = tmp[:-1]
        tmp = s*phi(tmp)
        cur_n_first_appr = cur_n_first_appr+tmp
        cur_n_first_appr = cur_n_first_appr+task_reg[:-1]
        cur_n_first_appr = (dt/(2*tau))*s*phi(cur_n_first_appr)
        
        #IV 7
        cur_n_next_task = (dt/(2*tau))*task_reg[1:]
        
        #All IVs in design matrix
        ext_des_mat = np.concatenate((cur_y.reshape(-1,1), other_ns_cur_spont.reshape(-1,1)), 1)
        ext_des_mat = np.concatenate((ext_des_mat, cur_y_phi.reshape(-1,1)), 1)
        ext_des_mat = np.concatenate((ext_des_mat, cur_n_task.reshape(-1,1)), 1)
        ext_des_mat = np.concatenate((ext_des_mat, other_ns_next_spont.reshape(-1,1)), 1)
        ext_des_mat = np.concatenate((ext_des_mat, cur_n_first_appr.reshape(-1,1)), 1)
        ext_des_mat = np.concatenate((ext_des_mat, cur_n_next_task.reshape(-1,1)), 1)

        ext_mod = sm.OLS(next_y, ext_des_mat)
        ext_res = ext_mod.fit()
        ext_params = ext_res.params

        ext_task_betas[region] = ext_params[6]
    
    return ext_task_betas



## Wrapper around network simulation and regression

In [ ]:
def sim_network_task_glm(ncommunities = 3, 
                         innetwork_dsity = .60, 
                         outnetwork_dsity = .08, 
                         hubnetwork_dsity = .25, 
                         nodespercommunity = 35, 
                         plot_network = False,
                         dt = 1, tau = 1, g = 1, s = 1, 
                         topdown = True, bottomup = False, 
                         local_com = 1, 
                         Tmax = 100000, 
                         plot_task = False, 
                         stimsize = np.floor(35/3.0), 
                         tasktiming = None, 
                         noise = 1):

    totalnodes = nodespercommunity*ncommunities

    # Construct structural matrix
    S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                        innetwork_dsity=innetwork_dsity,
                                        outnetwork_dsity=outnetwork_dsity,
                                        hubnetwork_dsity=hubnetwork_dsity,
                                        nodespercommunity=nodespercommunity,
                                        showplot=plot_network)
    # Construct synaptic matrix
    W = model.generateSynapticNetwork(S, showplot=plot_network)

    if plot_network:
        plt.rcParams["figure.figsize"][0] = 5
        plt.rcParams["figure.figsize"][1] = 4
        sns.heatmap(W, xticklabels=False, yticklabels=False)
        plt.xlabel('Regions')
        plt.ylabel('Regions')
        plt.title("Synaptic Weight Matrix -- Coupling Matrix")

    T = np.arange(0,Tmax,dt)

    # Construct timing array for convolution -- this timing is irrespective of the task being performed
    # Tasks are only determined by which nodes are stimulated
    if tasktiming is None:
        tasktiming = np.zeros((1,len(T)))
        for t in range(len(T)):
            if t%2000>500 and t%2000<1000:
                tasktiming[0,t] = 1.0

    if plot_task:
        if len(T)>9999:
            plt.plot(T[:10000], tasktiming[0,:10000])
            plt.ylim(top = 1.2, bottom = -0.1)
        else:
            plt.plot(T, tasktiming[0,:])
            plt.ylim(top = 1.2, bottom = -0.1)

    stimtimes = np.zeros((totalnodes,len(T)))

    # Construct a community affiliation vector
    Ci = np.repeat(np.arange(ncommunities),nodespercommunity) 

    if topdown:
        # Identify the regions associated with the hub network (hub network is by default the 0th network)
        hub_ind = np.where(Ci==0)[0] 
        stim_nodes = np.arange(0, stimsize,dtype=int)
    
    if bottomup:
        # Identify indices for one of the local communities
        local_ind = np.where(Ci==local_com)[0] 
        # Identify efferent connections from local network to hub network
        W_mask = np.zeros((W.shape))
        W_mask[local_ind,hub_ind] = 1.0
        local2hub_connects = np.multiply(W,W_mask)
        local_regions_wcon = np.where(local2hub_connects!=0)[0]
        local_regions_ncon = np.setdiff1d(local_ind,local_regions_wcon)
        #Half of the stimulated local community nodes have hub connections while the other does not
        stim_nodes = np.hstack((np.random.choice(local_regions_ncon, int(np.floor(stimsize/2)), replace=False),
                                np.random.choice(local_regions_wcon, int(stimsize-np.floor(stimsize/2)), replace=False)))
    
    # When task is ON the activity for a stim_node at that time point is .5
    for t in range(len(T)):
        if tasktiming[0,t] == 1:
            stimtimes[stim_nodes,t] = .5

    out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau, I=stimtimes, noise=noise)
    taskdata = out[0]

    short_lim = np.floor(taskdata.shape[1]/3)
    y = copy.copy(taskdata[:,:short_lim])

    ucr_glm = run_ucr_glm(all_nodes_ts = y, task_reg = I[stim_nodes[0],:])
    ext_glm = run_ext_glm(all_nodes_ts = y, task_reg = I[stim_nodes[0],:], 
                          weight_matrix = W, dt = dt, tau = tau, g = g, s = s)
    
    return(W, ucr_glm, ext_glm)


## Testing different network structures

Above we tested if true task activity can be recovered when accounting for connectivity in a neural network with one hub and two local networks.

Various parameters control the structure of this network and they can be maniupulated 

Thing to modulate:   
- hub and local network density  
- local ($s$) and global ($g$) information transfer strength (change in conjunction)  
- time step ($\tau$)
- number local networks   


### Changing network density

#### Increased innetwork density

In [ ]:
# Increased innetwork density
#innetwork_dsity = .60
innetwork_dsity = .80
#outnetwork_dsity = .08
#hubnetwork_dsity = .25

# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

plt.rcParams["figure.figsize"][0] = 5
plt.rcParams["figure.figsize"][1] = 4
sns.heatmap(W, xticklabels=False, yticklabels=False)
plt.xlabel('Regions')
plt.ylabel('Regions')
plt.title("Synaptic Weight Matrix -- Coupling Matrix")

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_innetwork = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_innetwork = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_innetwork, color = "blue", label = "Classic GLM (I)")
plt.plot(ext_glm_high_innetwork, color = "green", label = "Extended GLM (I)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

#### Increased outnetwork density

In [ ]:
# Increased innetwork density
innetwork_dsity = .60
#outnetwork_dsity = .08
outnetwork_dsity = .15
#hubnetwork_dsity = .25


# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

plt.rcParams["figure.figsize"][0] = 5
plt.rcParams["figure.figsize"][1] = 4
sns.heatmap(W, xticklabels=False, yticklabels=False)
plt.xlabel('Regions')
plt.ylabel('Regions')
plt.title("Synaptic Weight Matrix -- Coupling Matrix")

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_outnetwork = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_outnetwork = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_outnetwork, color = "blue", label = "Classic GLM (O)")
plt.plot(ext_glm_high_outnetwork, color = "green", label = "Extended GLM (O)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

#### Increased hub network density

Increasing the hub network density might make the overestimation of the uncorrected GLM parameters for the non-stimulated nodes worse but how well it is can be corrected with eGLM doesn't seem affected.

In [ ]:
# Increased innetwork density
#innetwork_dsity = .60
outnetwork_dsity = .08
#hubnetwork_dsity = .25
hubnetwork_dsity = .45


# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

plt.rcParams["figure.figsize"][0] = 5
plt.rcParams["figure.figsize"][1] = 4
sns.heatmap(W, xticklabels=False, yticklabels=False)
plt.xlabel('Regions')
plt.ylabel('Regions')
plt.title("Synaptic Weight Matrix -- Coupling Matrix")

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_hubnetwork = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_hubnetwork = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_hubnetwork, color = "blue", label = "Classic GLM (H)")
plt.plot(ext_glm_high_hubnetwork, color = "green", label = "Extended GLM (H)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

### Changing information transfer strength

#### Increasing $g$

In [ ]:
innetwork_dsity = .60
outnetwork_dsity = .08
hubnetwork_dsity = .25

# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=3,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_g = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_g = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_g, color = "blue", label = "Classic GLM (g)")
plt.plot(ext_glm_high_g, color = "green", label = "Extended GLM (g)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

#### Increasing $s$

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=20,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_s = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_s = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_s, color = "blue", label = "Classic GLM (s)")
plt.plot(ext_glm_high_s, color = "green", label = "Extended GLM (s)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

### Changing $\tau$

#### Increasing $\tau$

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=2,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_t = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_t = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_t, color = "blue", label = "Classic GLM (t)")
plt.plot(ext_glm_high_t, color = "green", label = "Extended GLM (t)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

#### Decreasing $\tau$

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=.5,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_t2 = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_t2 = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_t2, color = "blue", label = "Classic GLM (t2)")
plt.plot(ext_glm_high_t2, color = "green", label = "Extended GLM (t2)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

### Changing number of local communities

In [ ]:
# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=4,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

plt.rcParams["figure.figsize"][0] = 5
plt.rcParams["figure.figsize"][1] = 4
sns.heatmap(W, xticklabels=False, yticklabels=False)
plt.xlabel('Regions')
plt.ylabel('Regions')
plt.title("Synaptic Weight Matrix -- Coupling Matrix")

In [ ]:
stimtimes[task] = np.zeros((4*nodespercommunity,len(T)))

# When task is ON the activity for a stim_node at that time point is .5
for t in range(len(T)):
    if tasktiming[0,t] == 1:
        stimtimes[task][stim_nodes,t] = .5

out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_n = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_n = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 10
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_n, color = "blue", label = "Classic GLM (n)")
plt.plot(ext_glm_high_n, color = "green", label = "Extended GLM (n)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.axvline(x=105,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

### Changing temporal resolution

#### Decreasing $dt$

In [ ]:
# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

In [ ]:
T = np.arange(0,Tmax,0.5)

tasktiming = np.zeros((1,len(T)))
for t in range(len(T)):
    if t%2000>500 and t%2000<1000:
        tasktiming[0,t] = 1.0
        
stimtimes[task] = np.zeros((ncommunities*nodespercommunity,len(T)))

# When task is ON the activity for a stim_node at that time point is .5
for t in range(len(T)):
    if tasktiming[0,t] == 1:
        stimtimes[task][stim_nodes,t] = .5

out = model.networkModel(W,Tmax=Tmax,dt=.5,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_low_dt = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_low_dt = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_low_dt, color = "blue", label = "Classic GLM (dt)")
plt.plot(ext_glm_low_dt, color = "green", label = "Extended GLM (dt)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.axvline(x=105,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

#### Increasing $dt$

In [ ]:
T = np.arange(0,Tmax,2)

tasktiming = np.zeros((1,len(T)))
for t in range(len(T)):
    if t%2000>500 and t%2000<1000:
        tasktiming[0,t] = 1.0
        
stimtimes[task] = np.zeros((ncommunities*nodespercommunity,len(T)))

# When task is ON the activity for a stim_node at that time point is .5
for t in range(len(T)):
    if tasktiming[0,t] == 1:
        stimtimes[task][stim_nodes,t] = .5

out = model.networkModel(W,Tmax=Tmax,dt=2,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_high_dt = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_high_dt = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(ucr_glm_high_dt, color = "blue", label = "Classic GLM (dt)")
plt.plot(ext_glm_high_dt, color = "green", label = "Extended GLM (dt)")
plt.plot(true_task_stim, color = "black", label = "True task activity")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.axvline(x=105,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

## Testing different task structures

The initial task was a top down task that stimulated nodes only in the hub network and trickled activity down to the other nodes.

Other task activations are possible.

Things to modulate:  
- Number of nodes stimulated
- Stimulating only local community
- Stimulating both hub and local community
- Magnitude of stimulation (different from 0.5)

### Changing number of stimulated nodes

In [ ]:
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)



In [ ]:
stimtimes = {}

topdown_only = range(1,2)
task = 1

taskcount = task-np.min(topdown_only)
stimsize = np.floor(nodespercommunity/2.0)
stim_nodes = np.arange((taskcount)*stimsize,(taskcount)*stimsize+stimsize,dtype=int)
stimtimes[task] = np.zeros((totalnodes,len(T)))

# When task is ON the activity for a stim_node at that time point is .5
for t in range(len(T)):
    if tasktiming[0,t] == 1:
        stimtimes[task][stim_nodes,t] = .5

stim_nodes

In [ ]:
out = model.networkModel(W,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)
taskdata = out[0]
noise = out[1]

y = copy.copy(taskdata[:,:15000])

ucr_glm_more_stimn = run_ucr_glm(all_nodes_ts = y, task_reg = I[0,:])
ext_glm_more_stimn = run_ext_glm(all_nodes_ts = y, task_reg = I[0,:], weight_matrix = W)


In [ ]:
new_true_task_stim = np.zeros(totalnodes)
new_true_task_stim[stim_nodes] = stim_mag

In [ ]:
plt.rcParams["figure.figsize"][0] = 8
plt.rcParams["figure.figsize"][1] = 6
plt.plot(ucr_task, color = "blue", alpha = 0.5, label = "Classic GLM")
plt.plot(ext_task, color = "green", alpha = 0.5, label = "Extended GLM")
plt.plot(true_task_stim, color = "black", alpha=.5, label = "True task activity")
plt.plot(ucr_glm_more_stimn, color = "blue", label = "Classic GLM (stimn)")
plt.plot(ext_glm_more_stimn, color = "green", label = "Extended GLM (stimn)")
plt.plot(new_true_task_stim, color = "black", label = "True task activity (stimn)")
plt.ylabel('Beta',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axvline(x=35,linewidth=2, color='gray', ls = "--")
plt.axvline(x=70,linewidth=2, color='gray', ls = "--")
plt.axvline(x=105,linewidth=2, color='gray', ls = "--")
plt.legend(loc="best")

### Stimulating only local community

In [ ]:
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)



In [ ]:
stimtimes = {}

stimsize = np.floor(nodespercommunity/3.0)
#stim_nodes = np.arange(0,stimsize,dtype=int)
stimtimes[task] = np.zeros((totalnodes,len(T)))

local_ind = np.where(Ci==1)[0] # Identify indices for one of the local communities
# Identify efferent connections from local network to hub network
W_mask = np.zeros((W.shape))
W_mask[local_ind,hub_ind] = 1.0
local2hub_connects = np.multiply(W,W_mask)
local_regions_wcon = np.where(local2hub_connects!=0)[0]
local_regions_ncon = np.setdiff1d(local_ind,local_regions_wcon)



In [ ]:
np.hstack((np.random.choice(local_regions_ncon, int(np.floor(stimsize/2)), replace=False),
np.random.choice(local_regions_wcon, int(stimsize-np.floor(stimsize/2)), replace=False)))

In [ ]:
# When task is ON the activity for a stim_node at that time point is .5
for t in range(len(T)):
    if tasktiming[0,t] == 1:
        stimtimes[task][stim_nodes,t] = .5

### Stimulating both hub and local community

### Changing magnitude of stimulation

## Conclusions

1. Accounting for whether and how the node is connected to other nodes improves the task estimates both stimulated and non-stimulated nodes
    - Is the over- and underestimation real? If so, what is causing it?
2. The improvement in task estimates when accounting for connectivity is NOT affected by:
    - Network density
    - Number of local communities
3. The improvement in task estimates when accounting for connectivity IS affected by:
    - Information transfer strength (both local and global)  
    - $\tau$